<a href="https://colab.research.google.com/github/cate6495/ethereum/blob/main/ethereum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"catherinesanda","key":"2462dc492fa0914a6449e42c2027e18e"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download -d vldmrweider/ethereum-balances

 99% 1.12G/1.13G [01:15<00:01, 9.97MB/s]
100% 1.13G/1.13G [01:15<00:00, 16.1MB/s]


In [7]:
from zipfile import ZipFile
file_name="ethereum-balances.zip"
with ZipFile(file_name, 'r') as  zip:
  zip.extractall()
  print('Done')

Done


In [8]:
! pip install dask[dataframe]

     |████████████████████████████████| 134 kB 7.5 MB/s 


In [9]:
import time

In [10]:
import dask.dataframe as dd
s=time.time()
dataset1 = dd.read_csv("all_eth_balances.csv")
e=time.time()
print("Dask Loading Time={}".format(e-s))

Dask Loading Time=0.13797831535339355


In [11]:
dataset1.head()

,account;balance
0,0x0000000000000000000000000000000000000000;0x1...
1,0x0000000000000000000000000000000000000001;0x2...
2,0x0000000000000000000000000000000000000002;0x2...
3,0x0000000000000000000000000000000000000003;0x2...
4,0x0000000000000000000000000000000000000004;0x4...


In [12]:
import pandas as pd

In [14]:
ChunkSize = 10000000
for chunk in pd.read_csv("all_eth_balances.csv", chunksize=ChunkSize):
  print (chunk.shape)
  print("="*1000)
  print(chunk.head(5))
  print("="*1000)
  break


(10000000, 1)
                                     account;balance
0  0x0000000000000000000000000000000000000000;0x1...
1  0x0000000000000000000000000000000000000001;0x2...
2  0x0000000000000000000000000000000000000002;0x2...
3  0x0000000000000000000000000000000000000003;0x2...
4  0x0000000000000000000000000000000000000004;0x4...
(10000000, 1)
                                     account;balance
0  0x0000000000000000000000000000000000000000;0x1...
1  0x0000000000000000000000000000000000000001;0x2...
2  0x0000000000000000000000000000000000000002;0x2...
3  0x0000000000000000000000000000000000000003;0x2...
4  0x0000000000000000000000000000000000000004;0x4...


In [15]:
%%writefile testutility.py
import logging 
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re

def read_config_file(file_name):
  with open(file_name,'r')as stream:
    try:
      return yaml.safe_load(stream)
    except yaml.YAMLError as exc:
      logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string


def col_header_val(df,table_config):
  
  df.columns = df.columns.str.lower()
  df.columns = df.columns.str.replace('[^\w]','_',regex=True)
  df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
  df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
  expected_col = list(map(lambda x: x.lower(), table_config['columns']))
  expected_col.sort()
  df.columns = list(map(lambda x : x.lower(),list(df.columns)))
  df = df.reindex(sorted(df.columns), axis=1)
  if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
    print("column name and column length validation passed")
    return 1
  else:
    print("column name and column length validaton failed")
    mismatched_columns_file = list(set(df.columns).difference(expected_col))
    print("Following File columns are not in the Yaml file", mismatched_columns_file)
    missing_Yaml_file = list(set(expected_col).difference(df.columns))
    print("Following Yaml columns are not in the file uploaded", missing_Yaml_file)
    logging.info(f'df columns:{df.columns}')
    logging.info(f'expected columns:{expected_col}')
    return 0




Writing testutility.py


In [16]:
%%writefile yamlfile.yaml
file_type: csv
dataset_name: testfile
file_name: all_eth_balances
table_name: edsurv
inbound_delimiter: ";"
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
     - account
     - balance

Writing yamlfile.yaml


In [18]:
import testutility as util
config_data = util.read_config_file("yamlfile.yaml")

In [19]:
config_data

{'columns': ['account', 'balance'],
 'dataset_name': 'testfile',
 'file_name': 'all_eth_balances',
 'file_type': 'csv',
 'inbound_delimiter': ';',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'table_name': 'edsurv'}

In [20]:
file_type = config_data['file_type']
source_file = "./"+config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file, config_data['inbound_delimiter'])
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,account,balance
0,0x0000000000000000000000000000000000000000,0x19ea9837680860b830d
1,0x0000000000000000000000000000000000000001,0x20da79dfad9014b9
2,0x0000000000000000000000000000000000000002,0x2619639a1890820
3,0x0000000000000000000000000000000000000003,0x2a596306c84ef7a
4,0x0000000000000000000000000000000000000004,0x404980177e599dd


In [22]:
config_data['inbound_delimiter']

';'

In [23]:
util.col_header_val(df, config_data)

column name and column length validation passed


1

In [24]:
print("columns of the Yaml are:", config_data['columns'])
print("columns of the File are:", df.columns)

columns of the Yaml are: ['account', 'balance']
columns of the File are: Index(['account', 'balance'], dtype='object')


In [25]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipeline

column name and column length validation passed
col validation passed


In [26]:
import csv

In [27]:
reader=csv.reader(open("all_eth_balances.csv","rU"), delimiter=';')
writer=csv.writer(open("etherium_balances.txt",'w'),delimiter='|')
writer.writerows(reader)
print('Delimiter successfully changed')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.


Delimiter successfully changed


In [28]:
import gzip
import shutil
with open('etherium_balances.txt','rb')as f_in:
  with gzip.open('etherium_balances.txt.gz','wb')as f_out:
    shutil.copyfileobj(f_in, f_out)

In [29]:
import os
size = os.path.getsize('all_eth_balances.csv')
print(size)

2461359518


In [30]:
print('Number of rows:',len(df))

Number of rows: 46636596


In [31]:
print('Number of columns:',len(df.columns))


Number of columns: 2
